In [ ]:
# =====================
# hybrid_transformer_BiLSTM_XGBoost_full_pipeline.py
# Penulis: Ghiffary
# Tujuan: pipeline peramalan dengan koreksi galat pemodelan
# Data: Business-day 2014-01-24 to 2025-06-24
# 1 endogenous (Price) + 17 exogenous dummies + engineered features
# =====================

import os, random
import numpy as np, pandas as pd
import optuna, shap, tensorflow as tf
import matplotlib.pyplot as plt, seaborn as sns
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from tensorflow.keras import Model, Input, regularizers
from tensorflow.keras.layers import (
    LSTM, Bidirectional, Dense, Dropout,
    LayerNormalization, MultiHeadAttention, Add
)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from xgboost import XGBRegressor
from scipy.stats import skew, kurtosis

# ───────────────────────────────────────────────────────────────────────────────
# 0. Reproducibility & Settings
# ───────────────────────────────────────────────────────────────────────────────
SEED = 86
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED); np.random.seed(SEED); tf.random.set_seed(SEED)
sns.set(style="whitegrid", rc={"figure.figsize": (12,6)})

In [ ]:
# ───────────────────────────────────────────────────────────────────────────────
# 1. Plot ACF & PACF 
# ───────────────────────────────────────────────────────────────────────────────
def plot_acf_pacf(df, lags=60):
    price = df['Price'].values
    fig, axes = plt.subplots(1, 2, figsize=(14,4))
    sm.graphics.tsa.plot_acf(price, lags=lags, ax=axes[0])
    axes[0].set_title('ACF of Price')
    sm.graphics.tsa.plot_pacf(price, lags=lags, ax=axes[1])
    axes[1].set_title('PACF of Price')
    plt.tight_layout(); plt.show()

In [ ]:
# ───────────────────────────────────────────────────────────────────────────────
# 2. Pemodelan Fitur Temporal
# ───────────────────────────────────────────────────────────────────────────────
def load_and_fe(path, lags=[7,22], mas=[7,22], fourier=[5,21]):
    df = pd.read_csv(path, parse_dates=['date'], index_col='date').sort_index()
    # lag features
    for lag in lags:
        df[f'lag_{lag}'] = df['Price'].shift(lag)
    # moving averages
    for w in mas:
        df[f'ma_{w}'] = df['Price'].rolling(w).mean()
    # fourier terms
    t = np.arange(len(df))
    for p in fourier:
        df[f'sin_{p}'] = np.sin(2*np.pi*t/p)
        df[f'cos_{p}'] = np.cos(2*np.pi*t/p)
    df.dropna(inplace=True)
    return df

In [ ]:
# ───────────────────────────────────────────────────────────────────────────────
# 3. Split Data 80:20
# ───────────────────────────────────────────────────────────────────────────────
def train_test_split_time(df, test_size=0.2):
    idx = int(len(df)*(1-test_size))
    return df.iloc[:idx], df.iloc[idx:]

In [ ]:
# ───────────────────────────────────────────────────────────────────────────────
# 4. Preprocessor + Windowing + Inverse Scaling = Transformer
# ───────────────────────────────────────────────────────────────────────────────
class TimeSeriesPreprocessor:
    def __init__(self, time_steps=1):
        self.ts = time_steps
        self.scaler_y = MinMaxScaler()
        self.scaler_x = StandardScaler()
    def fit_transform(self, df):
        y = df[['Price']].values; X = df.drop(columns=['Price']).values
        y_s = self.scaler_y.fit_transform(y)
        X_s = self.scaler_x.fit_transform(X)
        return y_s, X_s
    def transform(self, df):
        y = df[['Price']].values; X = df.drop(columns=['Price']).values
        return self.scaler_y.transform(y), self.scaler_x.transform(X)
    def create_sequences(self, y_s, X_s):
        Ys, Xp, Xe = [], [], []
        for i in range(self.ts, len(y_s)):
            Ys.append(y_s[i,0])
            Xp.append(y_s[i-self.ts:i,0])
            Xe.append(X_s[i-self.ts:i,:])
        return np.expand_dims(np.array(Xp),-1), np.array(Xe), np.array(Ys)
    def inv_y(self, y_s):
        return self.scaler_y.inverse_transform(y_s.reshape(-1,1)).flatten()

In [ ]:
# ───────────────────────────────────────────────────────────────────────────────
# 5. Positional Encoding Layer = Transformer
# ───────────────────────────────────────────────────────────────────────────────
class PositionalEncoding(tf.keras.layers.Layer):
    def __init__(self, length, depth):
        super().__init__()
        pos = np.arange(length)[:,None]
        i   = np.arange(depth)[None,:]
        angle = pos/np.power(10000,(2*(i//2))/depth)
        angle[:,0::2] = np.sin(angle[:,0::2])
        angle[:,1::2] = np.cos(angle[:,1::2])
        self.pe = tf.constant(angle[None,...],dtype=tf.float32)
    def call(self, x):
        return x + self.pe[:,:tf.shape(x)[1],:]

In [ ]:
# ───────────────────────────────────────────────────────────────────────────────
# 6. Transformer-BiLSTM Base Model
# ───────────────────────────────────────────────────────────────────────────────
def build_transformer_bilstm(ts, feat_dim, p, n_blocks=3):
    inp_y = Input((ts,1)); inp_x = Input((ts,feat_dim))
    x = tf.concat([inp_y, inp_x],axis=-1)
    x = PositionalEncoding(ts, x.shape[-1])(x)
    for _ in range(n_blocks):
        x = Bidirectional(LSTM(p['lstm_units'], return_sequences=True,
                               dropout=p['dropout'],
                               kernel_regularizer=regularizers.l2(p['wd'])))(x)
        x = LayerNormalization()(x)
        att = MultiHeadAttention(num_heads=p['n_heads'], key_dim=p['key_dim'],
                                 dropout=p['attn_dropout'])(x,x)
        x = Add()([x, att]); x = LayerNormalization()(x)
        h = Dense(p['dense_units'], activation=p['activation'],
                  kernel_regularizer=regularizers.l2(p['wd']))(x)
        proj = Dense(x.shape[-1])(h)
        proj = Dropout(p['dense_dropout'])(proj)
        x = Add()([x, proj]); x = LayerNormalization()(x)
    x = tf.reduce_mean(x, axis=1)
    out = Dense(1)(x)
    m = Model([inp_y, inp_x], out)
    m.compile(Adam(p['lr']), loss='huber_loss', metrics=['mape','mae'])
    return m

In [ ]:
# ───────────────────────────────────────────────────────────────────────────────
# 7. Hyperparameter Tuning Base Model (Optuna + Pruner)
# ───────────────────────────────────────────────────────────────────────────────
def tune_base(Xp, Xe, y, trials=25):
    def obj(trial):
        p = {
            'lstm_units': trial.suggest_int('lstm_units',64,256),
            'dropout':    trial.suggest_float('dropout',0.1,0.6),
            'wd':         trial.suggest_float('wd',1e-6,1e-3,log=True),
            'n_heads':    trial.suggest_int('n_heads',2,8),
            'key_dim':    trial.suggest_int('key_dim',4,32),
            'attn_dropout':trial.suggest_float('attn_dropout',0.0,0.6),
            'dense_units':trial.suggest_int('dense_units',32,128),
            'dense_dropout':trial.suggest_float('dense_dropout',0.1,0.6),
            'activation': trial.suggest_categorical('activation',['relu','selu']),
            'lr':         trial.suggest_float('lr',1e-5,1e-2,log=True)
        }
        tr,vl = train_test_split(np.arange(len(y)), test_size=0.2, shuffle=False)
        m = build_transformer_bilstm(Xp.shape[1], Xe.shape[2], p)
        m.fit([Xp[tr],Xe[tr]], y[tr],
              validation_data=([Xp[vl],Xe[vl]], y[vl]),
              epochs=100, batch_size=64,
              callbacks=[
                  EarlyStopping(monitor='val_mape',patience=10,restore_best_weights=True),
                  ReduceLROnPlateau(monitor='val_mape',patience=5,factor=0.5)
              ], verbose=0)
        return float(m.history.history['val_mape'][-1])
    study = optuna.create_study(direction='minimize', pruner=optuna.pruners.MedianPruner())
    study.optimize(obj, n_trials=trials)
    return study.best_params

In [ ]:
# ───────────────────────────────────────────────────────────────────────────────
# 8. Train Base Model on Full Train Set (dengan validasi internal)
# ───────────────────────────────────────────────────────────────────────────────
def train_base(train_df, pre, best_params, n_blocks=3, val_split=0.1):
    # Preprocess
    y_s, X_s = pre.fit_transform(train_df)
    Xp, Xe, y_seq = pre.create_sequences(y_s, X_s)

    # Internal split
    split = int(len(y_seq)*(1-val_split))
    Xp_tr, Xp_val = Xp[:split], Xp[split:]
    Xe_tr, Xe_val = Xe[:split], Xe[split:]
    y_tr,  y_val  = y_seq[:split], y_seq[split:]

    model = build_transformer_bilstm(Xp.shape[1], Xe.shape[2], best_params, n_blocks)
    model.fit(
        [Xp_tr, Xe_tr], y_tr,
        validation_data=([Xp_val, Xe_val], y_val),   # <- sekarang tersedia val_*
        epochs=200, batch_size=64,
        callbacks=[
            EarlyStopping(monitor='val_mape', patience=20, restore_best_weights=True),
            ReduceLROnPlateau(monitor='val_mape', patience=10, factor=0.5)
        ],
        verbose=1
    )
    return model


In [ ]:
# ───────────────────────────────────────────────────────────────────────────────
# 9. Residual Feature Engineering 
# ───────────────────────────────────────────────────────────────────────────────
def get_residuals_and_feats(model, df, pre):
    # 1) Transform & sequences
    y_s, X_s = pre.transform(df)
    Xp, Xe, y_seq = pre.create_sequences(y_s, X_s)
    # 2) Base prediction & residual
    y_base_s = model.predict([Xp, Xe]).flatten()
    res_s    = y_seq - y_base_s

    # 3) Build rolling features
    df_r = pd.DataFrame({'res':res_s, 'pred':y_base_s})
    df_r['rm']   = df_r['res'].rolling(5).mean().fillna(0)
    df_r['rs']   = df_r['res'].rolling(5).std().fillna(0)
    df_r['skew'] = df_r['res'].rolling(5).apply(lambda x: skew(x), raw=True).fillna(0)
    df_r['kurt'] = df_r['res'].rolling(5).apply(lambda x: kurtosis(x), raw=True).fillna(0)

    last_y = Xp[:, -1, 0].reshape(-1,1)
    last_x = Xe[:, -1, :]
    X_res  = np.hstack([last_y, last_x, df_r[['pred','rm','rs','skew','kurt']].values])

    return y_seq, y_base_s, res_s, X_res

In [ ]:
# ───────────────────────────────────────────────────────────────────────────────
# 10. Train XGBoost on Residuals + Plot Feature Importance
# ───────────────────────────────────────────────────────────────────────────────
from xgboost import XGBRegressor
from sklearn.model_selection import RandomizedSearchCV
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# 10a. Train residual model dengan hyperparameter tuning yang lebih luas
def train_residual_xgb(X, y, trials=50):
    """
    Train XGBRegressor untuk memodelkan residual dengan RandomizedSearchCV.

    Parameters:
        X: np.array, residual features
        y: np.array, target residual (scaled)
        trials: int, jumlah iterasi pencarian hyperparameter

    Returns:
        best_model: trained XGBRegressor
    """
    param_dist = {
        'n_estimators': [100, 200, 300],
        'max_depth': [3, 4, 5, 6],
        'learning_rate': [0.005, 0.01, 0.05, 0.1],
        'subsample': [0.6, 0.7, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'gamma': [0, 1, 5],
        'reg_alpha': [0, 0.01, 0.1],
        'reg_lambda': [0.1, 1.0, 5.0]
    }

    xgb = XGBRegressor(objective='reg:squarederror', n_jobs=-1, verbosity=0)
    search = RandomizedSearchCV(
        xgb,
        param_distributions=param_dist,
        n_iter=trials,
        scoring='neg_root_mean_squared_error',
        cv=3,
        verbose=1,
        random_state=42
    )
    search.fit(X, y)

    print("✅ Best XGBoost Parameters:")
    for k, v in search.best_params_.items():
        print(f"  - {k}: {v}")
    return search.best_estimator_

# 10b. Advanced plot of XGBoost feature importances
def plot_xgb_importance(model, feature_names, top_n=20):
    """
    Plot top N feature importances from trained XGBoost model using SHAP-style barplot.

    Parameters:
        model: trained XGBRegressor
        feature_names: list of str, feature names
        top_n: int, number of top features to show
    """
    booster = model.get_booster()
    fmap = booster.get_score(importance_type='gain')

    # Convert to DataFrame
    imp_df = pd.DataFrame.from_dict(fmap, orient='index', columns=['gain'])
    imp_df.index.name = 'feature'
    imp_df = imp_df.reset_index()
    imp_df['index'] = imp_df['feature'].str.extract(r'f(\d+)').astype(int)
    imp_df['label'] = imp_df['index'].apply(lambda i: feature_names[i] if i < len(feature_names) else f'f{i}')
    imp_df = imp_df.sort_values(by='gain', ascending=False).head(top_n)

    # Plot
    plt.figure(figsize=(10, 6))
    sns.barplot(data=imp_df, x='gain', y='label', palette='rocket')
    plt.title(f"Top {top_n} Most Important Features (by Gain)", fontsize=14, weight='bold')
    plt.xlabel("Average Gain", fontsize=12)
    plt.ylabel("Features", fontsize=12)
    plt.grid(axis='x', linestyle='--', alpha=0.5)
    plt.tight_layout()
    plt.show()

In [ ]:
# ───────────────────────────────────────────────────────────────────────────────
# 11. Evaluate & Plot (Train/Test) — inverse scale & MAPE 
# ───────────────────────────────────────────────────────────────────────────────
def evaluate_and_plot(base_m, res_m, train_df, test_df, pre):
    # TRAIN
    y_tr_s, yb_tr_s, res_tr_s, Xr_tr = get_residuals_and_feats(base_m, train_df, pre)
    yres_tr_s = res_m.predict(Xr_tr)
    yh_tr_s   = yb_tr_s + yres_tr_s

    # TEST
    y_te_s, yb_te_s, res_te_s, Xr_te = get_residuals_and_feats(base_m, test_df, pre)
    yres_te_s = res_m.predict(Xr_te)
    yh_te_s   = yb_te_s + yres_te_s

    # INVERSE to original scale
    y_tr,  yb_tr,  yres_tr,  yh_tr  = pre.inv_y(y_tr_s),  pre.inv_y(yb_tr_s), pre.inv_y(yres_tr_s), pre.inv_y(yh_tr_s)
    y_te,  yb_te,  yres_te,  yh_te  = pre.inv_y(y_te_s),  pre.inv_y(yb_te_s), pre.inv_y(yres_te_s), pre.inv_y(yh_te_s)

    from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
    def calc_metrics(a, p):
        return {
            'MAPE': np.mean(np.abs((a-p)/a))*100,
            'RMSE': np.sqrt(mean_squared_error(a,p)),
            'MAE':  mean_absolute_error(a,p),
            'R2':   r2_score(a,p)
        }

    m_tr = calc_metrics(y_tr, yh_tr)
    m_te = calc_metrics(y_te, yh_te)
    print("Train metrics:", m_tr)
    print("Test  metrics:", m_te)

    plt.figure(figsize=(12,5))
    plt.plot(y_te,        label='Actual', color='black', lw=1)
    plt.plot(yb_te, '--',  label='Base', alpha=0.7)
    plt.plot(yres_te, ':', label='XGB Residual', alpha=0.7)
    plt.plot(yh_te, '-.',  label='Hybrid', color='C3', lw=2)
    plt.title(f"Test Set Comparison\nMAPE={m_te['MAPE']:.2f}%, RMSE={m_te['RMSE']:.2f}, MAE={m_te['MAE']:.2f}")
    plt.xlabel('Time Index'); plt.ylabel('Price')
    plt.legend(); plt.grid(alpha=0.3); plt.tight_layout(); plt.show()

    combined_actual = np.concatenate([y_tr, y_te])
    combined_hybrid = np.concatenate([yh_tr, yh_te])
    sep = len(y_tr)

    plt.figure(figsize=(12,5))
    plt.plot(combined_actual, label='Actual', color='black', lw=1)
    plt.plot(combined_hybrid, '--', label='Hybrid', color='C3', lw=1.5)
    plt.axvline(sep, color='red', linestyle=':', lw=2)
    plt.text(sep+1, combined_actual.mean(), 'Test Start', color='red')
    plt.title("Combined Train & Test Hybrid Forecast")
    plt.xlabel('Time Index'); plt.ylabel('Price')
    plt.legend(); plt.grid(alpha=0.3); plt.tight_layout(); plt.show()

In [ ]:
# ───────────────────────────────────────────────────────────────────────────────
# 12. Forecast 
# ───────────────────────────────────────────────────────────────────────────────
def forecast_full_and_save(base_m, res_m, df_full, pre, prefix='hybrid'):
    # 1) Prepare sequences & predictions
    y_s, X_s = pre.transform(df_full)
    Xp, Xe, y_seq = pre.create_sequences(y_s, X_s)
    yb_s  = base_m.predict([Xp, Xe]).flatten()
    _, _, _, Xr_full = get_residuals_and_feats(base_m, df_full, pre)
    yres_s = res_m.predict(Xr_full)
    yh_s   = yb_s + yres_s

    # 2) Inverse scale
    y_o  = pre.inv_y(y_seq)
    yh_o = pre.inv_y(yh_s)

    # 3) 95% CI using rolling std of training residuals
    resid = y_o - yh_o
    std_rolling = pd.Series(resid).rolling(window=30).std().bfill().values
    lower = yh_o - 1.96 * std_rolling
    upper = yh_o + 1.96 * std_rolling

    # 4) Plot with CI
    plt.figure(figsize=(12,5))
    plt.plot(y_o,      label='Actual', color='black', lw=1)
    plt.plot(yh_o, '--',label='Hybrid Forecast', color='C3', lw=1.5)
    plt.fill_between(range(len(yh_o)), lower, upper,
                     color='C3', alpha=0.2, label='95% CI')
    plt.title('Full Data Hybrid Forecast with 95% CI')
    plt.xlabel('Time Index'); plt.ylabel('Price')
    plt.legend(); plt.grid(alpha=0.3); plt.tight_layout(); plt.show()

    # 5) Save
    #base_m.save(f'{prefix}_base.h5')
    #import joblib
    #joblib.dump(res_m, f'{prefix}_xgb.pkl')
    #joblib.dump(pre, f'{prefix}_pre.pkl')

In [ ]:
if __name__ == '__main__':
    import logging
    logging.basicConfig(level=logging.INFO)

    # 1. Load data & ACF PACF
    df_full = load_and_fe('final_data_with_dummy_anomaly.csv')
    plot_acf_pacf(df_full, lags=60)

    # 2. time_steps
    TIME_STEPS = 22
    logging.info(f"Using TIME_STEPS = {TIME_STEPS}")

    # 3. Split Train/Test
    train_df, test_df = train_test_split_time(df_full, test_size=0.2)
    logging.info(f"Train size = {len(train_df)}, Test size = {len(test_df)}")

    # 4. Preprocessing
    preproc = TimeSeriesPreprocessor(time_steps=TIME_STEPS)
    y_s, X_s = preproc.fit_transform(train_df)
    Xp, Xe, y = preproc.create_sequences(y_s, X_s)

    # 5. Tuning & train base model (Transformer-BiLSTM)
    best_params = tune_base(Xp, Xe, y, trials=5)
    base_model = train_base(train_df, preproc, best_params, n_blocks=3)

    # 6. Residual dan Fitur
    y_tr_s, yb_tr_s, res_tr_s, Xr_tr = get_residuals_and_feats(base_model, train_df, preproc)

    # 7. Train XGBoost pada Residual
    xgb_model = train_residual_xgb(Xr_tr, res_tr_s, trials=25)

    # 8. Plot XGB feature importance
    feat_names = ['last_price'] + list(train_df.drop(columns=['Price']).columns) + ['pred', 'rm', 'rs', 'skew', 'kurt']
    plot_xgb_importance(xgb_model, feat_names, top_n=15)

    # SHAP summary plot
    import shap
    explainer = shap.Explainer(xgb_model)
    shap_values = explainer(Xr_tr[:500])
    shap.summary_plot(shap_values, Xr_tr[:500], feature_names=feat_names, max_display=15)

    # 9. Evaluasi Model
    evaluate_and_plot(base_model, xgb_model, train_df, test_df, preproc)

    # 10. Forecast
    forecast_full_and_save(base_model, xgb_model, df_full, preproc, prefix='hybrid_model')